In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [6]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-8a', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
11,CRWD,RROnly,1825.61,1514.20,3339.81,37.893874,0.952128,150.49,183.830002,0.22,9.303675
35,MNDY,RROnly,977.06,1900.44,2877.51,84.547460,0.937500,160.99,204.380005,0.27,9.143519
62,TTD,RROnly,744.64,512.43,1257.07,16.256295,0.936170,84.84,101.529999,0.20,9.042553
40,NVCR,RROnly,1315.10,1036.43,2351.54,31.923473,0.914894,28.55,41.029999,0.44,8.955513
59,TEAM,RROnly,2135.95,2272.32,4408.27,113.716257,0.946809,198.84,178.710007,-0.10,8.936170
61,TOST,RROnly,65.00,119.00,184.00,13.541693,0.914634,20.02,27.809999,0.39,8.927455
64,U,RROnly,422.04,937.26,1359.30,24.279663,0.916667,40.17,53.730000,0.34,8.921569
19,ESTC,RROnly,950.31,1062.48,2012.79,46.878122,0.925532,63.51,75.870003,0.19,8.863382
24,GLBE,RROnly,188.85,292.63,481.48,18.496157,0.940000,41.13,45.480000,0.11,8.854545
14,DDOG,RROnly,1068.20,928.47,1996.67,31.590757,0.923497,107.53,123.330002,0.15,8.724954


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-7a', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
2,ARKW,RROnly,805.91,786.66,1592.57,38.158114,0.984043,57.96,65.870003,0.14,9.726444
0,ARKF,RROnly,261.97,224.44,486.41,14.147363,0.962766,21.06,23.840000,0.13,9.341244
23,SMOG,RROnly,873.66,661.55,1535.21,51.864062,0.957447,118.43,133.539993,0.13,9.247136
30,XITK,RROnly,1008.97,827.19,1836.16,86.468085,0.952128,128.30,141.190002,0.10,9.042553
10,IPO,RROnly,351.14,314.69,665.83,22.307335,0.946809,32.46,35.830002,0.10,8.936170
43,XNTK,RROnly,832.40,391.70,1224.10,64.272084,0.968085,138.26,143.940002,0.04,8.882979
20,ONLN,RROnly,281.45,384.87,666.32,27.105897,0.930851,34.85,36.840000,0.06,8.156028
7,FPX,RROnly,472.74,544.47,1017.21,55.536532,0.946809,87.01,90.330002,0.04,8.138298
11,IXC,RROnly,133.59,126.75,260.34,11.540287,0.898936,40.37,36.320000,-0.10,7.978723
17,MGK,RROnly,1040.64,666.63,1707.26,111.184990,0.952128,232.96,226.830002,-0.03,7.925532


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
             'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4a', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
34,OMQS,RROnly,71.85,85.18,157.02,3.066616,0.920213,2.81,3.74,0.33,5.171237
10,BSGM,RROnly,39.72,65.57,105.29,0.463700,0.898936,0.62,1.03,0.67,5.137018
33,NSTG,RROnly,381.74,369.87,751.61,4.694782,0.872340,3.07,4.92,0.60,4.918648
1,APRN,RROnly,1163.95,1697.15,2861.09,8.363324,0.877660,6.22,8.22,0.32,4.780390
35,ONDS,RROnly,42.06,68.96,111.03,1.119602,0.869919,1.14,1.54,0.36,4.755832
36,OPEN,RROnly,112.05,125.01,237.06,1.585639,0.856164,3.43,4.97,0.45,4.716626
42,RIG,RROnly,56.88,49.13,106.01,0.840682,0.898936,8.60,7.13,-0.17,4.693367
38,OUST,RROnly,445.33,601.49,1046.82,6.124946,0.861538,6.61,8.96,0.36,4.683258
48,YEXT,RROnly,97.13,110.32,207.45,4.467473,0.882979,8.85,10.70,0.21,4.636748
4,AMSC,RROnly,141.72,118.79,260.51,3.812342,0.840426,8.77,12.94,0.48,4.611233


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5a', mod4, False)[0]

In [9]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
18,FDX,RROnly,1403.76,1272.49,2676.25,89.574832,0.930851,262.65,284.609985,0.08,5.784574
30,PYPL,RROnly,943.30,1342.67,2285.97,63.066485,0.856383,59.42,70.010002,0.18,5.319149
28,NOW,RROnly,3405.43,2578.90,5984.33,231.001559,0.914894,541.50,574.179993,0.06,5.116515
15,DHR,RROnly,1342.12,919.18,2261.30,123.507453,0.898936,251.41,268.649994,0.07,4.977204
31,QCOM,RROnly,579.12,648.67,1227.79,60.165653,0.845745,110.10,123.709999,0.12,4.755572
4,AMZN,RROnly,821.53,628.36,1449.89,84.261756,0.882979,133.22,142.020004,0.07,4.635258
17,GOOGL,RROnly,482.54,300.94,783.48,54.788010,0.882979,127.46,136.509995,0.07,4.635258
34,TSLA,RROnly,2707.93,1514.35,4222.28,23.111982,0.840426,215.49,237.080002,0.10,4.407295
2,ADBE,RROnly,3120.17,1568.09,4688.26,272.326920,0.914894,508.13,530.739990,0.04,4.407295
11,CRM,RROnly,1694.47,1121.25,2815.72,128.554291,0.882979,204.83,216.639999,0.06,4.356636


In [10]:
findata.EPOCHS=200
tickers5 = ['AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-8b', mod5, True)[0]


/usr/local/lib/python3.10/dist-packages/yahoo_fin/stock_info.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['ticker'] = ticker.upper()


Epoch 1/200
773/776 [============================>.] - ETA: 0s - loss: 0.0057 - mean_absolute_error: 0.0787
Epoch 1: val_loss improved from inf to 0.00415, saving model to results/gstock-8b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
776/776 [==============================] - 17s 12ms/step - loss: 0.0057 - mean_absolute_error: 0.0787 - val_loss: 0.0042 - val_mean_absolute_error: 0.0653
Epoch 2/200
772/776 [============================>.] - ETA: 0s - loss: 0.0047 - mean_absolute_error: 0.0727
Epoch 2: val_loss improved from 0.00415 to 0.00409, saving model to results/gstock-8b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
776/776 [==============================] - 9s 12ms/step - loss: 0.0047 - mean_absolute_error: 0.0726 - val_loss: 0.0041 - val_mean_absolute_error: 0.0669
Epoch 3/200
776/776 [==============================] - ETA: 0s - loss: 0.0044

/usr/local/lib/python3.10/dist-packages/runml/pipeline.py:249: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


1/1 [==============================] - 0s 31ms/step


/usr/local/lib/python3.10/dist-packages/runml/pipeline.py:249: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


...

/usr/local/lib/python3.10/dist-packages/runml/pipeline.py:249: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


1/1 [==============================] - 0s 30ms/step


/usr/local/lib/python3.10/dist-packages/runml/pipeline.py:249: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


1/1 [==============================] - 0s 50ms/step


/usr/local/lib/python3.10/dist-packages/runml/pipeline.py:249: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(


In [11]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
55,SE,RROnly,1562.12,1299.95,2862.07,32.695374,0.904255,38.68,62.689999,0.62,6.150113
5,ADYEY,RROnly,147.68,99.45,247.13,6.809365,0.874317,9.60,15.000000,0.56,5.796253
42,NU,RROnly,20.82,28.32,49.14,3.485500,0.931507,7.21,7.920000,0.10,5.283757
20,EQNR,RROnly,152.43,131.89,284.32,7.719486,0.936170,30.39,32.750000,0.08,5.091185
33,MELI,RROnly,13185.21,10397.79,23583.00,479.109675,0.920213,1199.59,1317.670044,0.10,4.977204
3,ASML,RROnly,3509.69,3954.41,7464.10,202.184278,0.914894,655.33,591.119995,-0.10,4.832827
10,COVTY,RROnly,125.75,139.38,265.12,12.058916,0.914894,26.05,23.660000,-0.09,4.643701
13,CX,RROnly,42.92,37.58,80.50,1.771175,0.898936,7.80,8.630000,0.11,4.583448
0,AEXAF,RROnly,221.53,497.02,718.55,8.785957,0.867021,8.00,9.280000,0.16,4.530775
58,TEF,RROnly,20.53,29.72,50.26,2.645464,0.952128,3.87,3.700000,-0.04,4.407295


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-7b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
13,EWZ,RROnly,175.01,194.36,369.37,16.708554,0.952128,33.02,30.840000,-0.07,8.837386
0,ARGT,RROnly,207.38,129.58,336.96,14.284427,0.914894,47.43,38.869999,-0.18,8.676123
15,ILF,RROnly,104.65,133.58,238.22,12.685837,0.930851,27.73,25.879999,-0.07,8.320669
16,KEMQ,RROnly,102.41,157.14,259.55,11.917421,0.909574,15.02,16.400000,0.09,8.091017
2,CUBS,RROnly,22.14,37.86,60.00,18.708547,0.924731,19.99,20.900000,0.05,7.741935
3,FLN,RROnly,89.67,86.26,175.93,8.884696,0.930851,19.57,18.879999,-0.04,7.579787
11,EWW,RROnly,227.75,130.06,357.80,23.868301,0.904255,62.90,59.299999,-0.06,7.446809
17,THD,RROnly,237.39,290.67,528.05,45.385048,0.920213,68.46,65.800003,-0.04,7.207447
8,EWC,RROnly,153.14,67.69,220.83,16.783446,0.914894,35.80,34.529999,-0.04,7.021277
4,FM,RROnly,77.02,67.95,144.97,18.109017,0.909574,27.62,26.600000,-0.04,6.835106


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ALB', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-6b', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
30,FMS,RROnly,139.51,203.56,343.07,12.876209,0.941489,26.28,28.340000,0.08,8.683511
38,JWN,RROnly,250.48,301.26,551.75,11.730722,0.925532,22.95,25.950001,0.13,8.682488
35,IP,RROnly,193.05,238.24,431.29,22.119775,0.930851,35.74,32.529999,-0.09,8.540189
56,RH,RROnly,4129.86,2681.53,6811.39,90.612399,0.941489,387.29,407.899994,0.05,8.244681
13,CF,RROnly,658.52,359.28,1017.80,20.758730,0.920213,81.23,88.070000,0.08,8.204787
69,URI,RROnly,2561.79,1531.32,4093.11,76.483470,0.882979,446.07,364.869995,-0.18,8.179669
57,SAFE,RROnly,551.31,548.51,1099.81,23.685274,0.909574,24.73,27.030001,0.09,8.091017
17,CG,RROnly,247.31,197.89,445.21,15.991252,0.936170,35.28,36.930000,0.05,8.085106
16,CIVI,RROnly,399.02,303.91,702.93,8.701244,0.904255,73.85,66.160004,-0.10,8.085106
34,HPE,RROnly,84.44,74.45,158.89,7.237447,0.920213,17.26,16.070000,-0.07,8.062310


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-7b', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
10,WEAT,RROnly,28.23,30.27,58.50,5.075308,0.813830,6.99,6.220000,-0.11,6.445841
1,ISHG,RROnly,80.80,90.42,171.22,63.693936,0.840426,70.42,76.279999,0.08,6.409574
8,UUP,RROnly,32.27,30.47,62.74,24.055419,0.861702,28.17,27.110001,-0.04,5.159574
4,SLV,RROnly,121.78,115.74,237.52,11.595939,0.909574,22.32,22.690001,0.02,4.574468
5,SOYB,RROnly,73.26,38.20,111.46,13.665571,0.829787,28.51,27.580000,-0.03,2.624113
0,GLD,RROnly,424.33,411.72,836.05,138.504898,0.888298,181.86,180.050003,-0.01,0.000000
3,PDBC,RROnly,39.26,37.42,76.68,6.679169,0.797872,14.74,14.750000,0.00,0.000000
6,TIP,RROnly,108.85,111.28,220.13,95.969435,0.861702,107.13,106.430000,-0.01,0.000000
7,TLT,RROnly,228.33,302.39,530.72,92.138539,0.845745,99.81,98.330002,-0.01,0.000000
9,VTIP,RROnly,27.86,14.34,42.21,41.126449,0.771277,47.26,47.080002,-0.00,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-6b', mod9, False)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
3,DOGE-USD,RROnly,2.48,2.25,4.73,0.013415,0.828877,0.08,0.060000,-0.21,5.105679
0,ADA-USD,RROnly,9.81,11.99,21.80,0.152873,0.786096,0.29,0.230000,-0.21,4.596384
2,DOT-USD,RROnly,272.78,275.29,548.07,5.562015,0.754011,4.94,4.020000,-0.19,4.091110
4,ETH-USD,RROnly,29404.31,30200.57,59604.88,561.650477,0.898396,1829.23,1899.319946,0.04,3.877005
6,MATIC-USD,RROnly,16.75,13.77,30.52,0.095509,0.850267,0.66,0.620000,-0.06,3.577795
5,LTC-USD,RROnly,1244.08,1935.09,3179.17,54.189480,0.802139,82.42,77.059998,-0.06,2.431882
1,BTC-USD,RROnly,262255.68,214176.35,476432.03,17246.473566,0.695187,28998.75,28499.130859,-0.02,0.000000
7,XRP-USD,RROnly,8.00,7.33,15.34,0.263007,0.796791,0.62,0.600000,-0.03,0.000000


In [ ]:
findata.EPOCHS=200
save = ['CF']
tickers10 = ['AAPL', 'AN', 'AZN', 'BJ', 'BLDR',
             'COST', 'CPRX', 'CROX', 'DHR', 'DLTR', 'DY',
             'ELF', 'EQNR',  'HRMY', 'HUM',  'LNTH',
             'MCK',  'MPWR', 'NVDA', 'ON', 'PWR', 'QQQ',  'SWAV',
             'TSLA', 'UNH', 'URI', 'VRRM', 'XLE']
mod10 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df10 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod10, False)[0]

loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df10, 0.07, 1)
df10.sort_values('Alloc', ascending=False)

In [ ]:
findata.G_LOOKUP_STEP = 5
mod11 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df11 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod11, False)[0]


loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-5-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df11, 0.07, 1)
df11.sort_values('Alloc', ascending=False)

In [12]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
gstock-8b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,590 bytes  received 35 bytes  6,443,250.00 bytes/sec
total size is 192,746,056  speedup is 59.83
/content/drive/MyDrive/colab/results/mcap-5a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-5a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-5a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-5a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
